In [1]:
from result_saver import SaverProvider
provider = SaverProvider()

In [ ]:
import numpy as np
# parameter grid
distances = [30, 40, 55]
rounds = np.arange(7, 41, 2) # UNGERADE ZAHLEN!!!
#rounds = np.append(rounds, [35, 35, 35, 35])
# rounds = rounds[::-1]
print(rounds)
# devices = ["ibm_nazca", "ibm_sherbrooke", "ibm_brisbane", "ibm_cusco"]
devices = ["ibm_torino"]
logicals = ['0', '1']

[ 7  9 11 13 15 17 19 21 23 25 27 29 31 33 35 37 39]


# Sligthly faster faster triple loop

In [ ]:
from tqdm import tqdm

from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit import transpile

from soft_info import find_longest_path_in_hex
from Scratch import metadata_helper


# hex_backend = provider.get_backend('ibm_sherbrooke')
# path, _, _  = find_longest_path_in_hex(hex_backend)

import pickle
with open('longest_path_torino.pkl', 'rb') as f:
    path = pickle.load(f)

for nb_of_rounds in tqdm(rounds):
    for distance in (distances):  
        bounded_path = path[:2 * distance - 1]
        layout = bounded_path[1::2] + bounded_path[::2] 
        code = RepetitionCodeCircuit(distance, nb_of_rounds) 
        for device in devices:
            backend = provider.get_backend(device)
            for logical in logicals:          
                qc = code.circuit[logical]                                   
                transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=3)

                n_shots = int(1e6 / (distance*nb_of_rounds)) #/ 10 # for torino (?)
                metadata = metadata_helper(descr='diff rounds v2', code="RepetitionCodeCircuit", distance=int(distance), rounds=f"{nb_of_rounds}",  logical=logical, layout='_is_hex=True, sherbrooke')
                backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', job_tags=["RepCode !/10 Opt = 3"]) # meas_level=1 for IQ  
                


  0%|          | 0/17 [00:00<?, ?it/s]

10:58:54 Warning: divide by zero encountered in det. IN FILE: /Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/numpy/linalg/linalg.py, LINE: 2180
10:58:54 Warning: invalid value encountered in det. IN FILE: /Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/numpy/linalg/linalg.py, LINE: 2180
10:58:55 Warning: updating metadata. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 46


  0%|          | 0/17 [00:16<?, ?it/s]


KeyboardInterrupt: 